These functions will help you extract the entire report and place them into a pandas dataframe - please remember to download the conversion table (dow 30 to cik #):
1) getting_crawler(year_number, qtr): this function will download the index file as provdied by sec.gov. Please indicate the year as well the quarter you would like to extract the data from. For more information, visit https://www.sec.gov/edgar/searchedgar/accessing-edgar-data.htm

2) milestone_search(file_name): milestone_search looks for where the listing begins in the index file.

3) convert_to_pandas(file_name, data_start): convert_to_pandas function converts index file to a pandas data frame

4) search_company_name(data, company_name, report_type): allows you to find the url for the report as well as the the specified companies. Please note that the function currently only take CIK# for accuracy. Many larger companies have multiple reporting sub-companies, i.e., J P Morgan.

5) getting_data(url_list, year_number, qtr, report_type): Will download the entire webpage of the report as a file and put it into a pandas dataframe. For missing values, it will automatically assigned np.NaN

6) adding_company_name(data): will add company name and other inforamtion back to the current dataframe

7) putting_together(year_number,qtr, company_name, report_type): goes through the entire process and only return the data_frame with the full data.

8) the functions now support saving and reading in pickle files!

See below listing for the options for company_name

In [417]:
import urllib
import pandas as pd
import struct
import numpy as np
from bs4 import BeautifulSoup
import requests
import regex as re
import pickle
import os.path
import datetime
import spacy
import string

In [4]:
sp500 = pd.read_csv("sp500.csv")

In [7]:
sp500.head(2)

,Ticker symbol,Security,SEC filings,GICS?Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,A,Agilent Technologies Inc,reports,Health Care,Health Care Equipment,"Santa Clara, California",2000-06-05,1090872
1,AAL,American Airlines Group,reports,Industrials,Airlines,"Fort Worth, Texas",2015-03-23,6201


In [10]:
new = sp500[["Ticker symbol","Security","CIK"]]

In [16]:
new = new.rename(columns={"Ticker symbol":"Company","Security":"Name","CIK":"CIK #"})

In [17]:
new.to_csv("sp500_new.csv")

In [2]:
company_table = pd.read_csv("cik_list.csv")

In [3]:
company_table.head(3)

,Company,Name,CIK #
0,AAPL,Apple Inc.,320193
1,AXP,American Expression Co.,4962
2,BA,Boeing Co.,12927


In [58]:
cik_lookup = pd.read_table("cik_look_up.txt",names = ["company"])

In [59]:
cik_lookup.head(5)

,company
0,!J INC:0001438823:
1,"#1 A LIFESAFER HOLDINGS, INC.:0001509607:"
2,#1 ARIZONA DISCOUNT PROPERTIES LLC:0001457512:
3,#1 PAINTBALL CORP:0001433777:
4,$ LLC:0001427189:


In [60]:
def clean_up(pattern, x):
    if not re.match(pattern, x):
        return np.NaN
    else:
        return re.match(pattern, x).group(1).lower()

In [61]:
cik_lookup.loc[:,"Name"] = cik_lookup["company"].apply(lambda x: clean_up("(.+):\d+:",x))

In [62]:
cik_lookup.loc[:,"cik #"] = cik_lookup["company"].apply(lambda x: clean_up(".+:(\d+):",x))

In [63]:
cik_lookup.head(3)

,company,Name,cik #
0,!J INC:0001438823:,!j inc,0001438823
1,"#1 A LIFESAFER HOLDINGS, INC.:0001509607:","#1 a lifesafer holdings, inc.",0001509607
2,#1 ARIZONA DISCOUNT PROPERTIES LLC:0001457512:,#1 arizona discount properties llc,0001457512


In [64]:
def stripping_0(x):
    if not x:
        return np.NaN
    else:
        return str(x).strip("0")

In [65]:
cik_lookup["CIK #"] = cik_lookup["cik #"].apply(lambda x: stripping_0(x))

In [67]:
cik_lookup.head(5)

,company,Name,cik #,CIK #
0,!J INC:0001438823:,!j inc,0001438823,1438823
1,"#1 A LIFESAFER HOLDINGS, INC.:0001509607:","#1 a lifesafer holdings, inc.",0001509607,1509607
2,#1 ARIZONA DISCOUNT PROPERTIES LLC:0001457512:,#1 arizona discount properties llc,0001457512,1457512
3,#1 PAINTBALL CORP:0001433777:,#1 paintball corp,0001433777,1433777
4,$ LLC:0001427189:,$ llc,0001427189,1427189


In [70]:
cik_lookup.shape

(661046, 4)

In [71]:
cik_nm = cik_lookup.dropna().reset_index(drop=True)

In [72]:
cik_nm.shape[0]

661042

In [73]:
cik_nm.to_csv("cik_nm.csv",index=False)
#cik_nm.to_pickle("cik_nm.txt")

In [19]:
def getting_crawler(year_number,qtr):
    base_url = "https://www.sec.gov/Archives/edgar/full-index/"
    year = str(year_number) + "/"
    if qtr == 1:
        QTR = "QTR1/"
    if qtr == 2:
        QTR = "QTR2/"
    if qtr == 3:
        QTR = "QTR3/"
    if qtr == 4:
        QTR = "QTR4/"
    download_file = "crawler.idx"
    full = base_url + year + QTR + download_file
    file_name = str(year_number) + "_QTR" + str(qtr) + "_crawler.idx"
    testfile = urllib.URLopener()
    testfile.retrieve(full, file_name)
    return file_name

In [20]:
def milestone_search(file_name):
    with open(file_name) as myFile:
        for num, line in enumerate(myFile, 0):
            if ("CIK" in line):
                if ("Form Type" in line):
                    #print 'found at line:', num
                    line_start = num
            if "------------------" in line:
                #print "found ----- at line:", num
                data_start = num+1
                #print "data starts at:", data_start
    return data_start

In [111]:
def convert_to_pandas(file_name, data_start):
    company_name = list()
    report_type = list()
    CIK = list()
    date = list()
    url = list()
    with open(file_name) as myfile:
        testing = myfile.readlines()[data_start:]
        for record in testing:
            record_split = map(lambda x: x.strip(), filter(lambda x: x!="" and x !="\n" and x != " \n", record.split("  ")))
            if len(record_split) < 5:
                continue
            else:
                company_name.append(record_split[0])
                report_type.append(record_split[1])
                CIK.append(record_split[2])
                date.append(record_split[3])
                url.append(record_split[4])
    data_dict = {"company":company_name,
                 "report":report_type,
                 "CIK":CIK,
                 "date":date,
                 "url":url}
    df = pd.DataFrame.from_dict(data_dict)
    df["company_l"] = df["company"].str.lower()
    return df

In [22]:
def search_company_name(data, company_name, report_type):
    data_report = data[data["report"] == report_type]
    company_table = pd.read_csv(company_name)
    company_table = company_table[["Name","CIK #"]]
    company_table["CIK #"] = company_table["CIK #"].astype(str)
    result = pd.merge(data_report, company_table, how ='inner', left_on = "CIK", right_on = "CIK #")
    result = result[["Name", "CIK #", "company", "report", "date", "url"]].reset_index(drop=True)
    return result

In [23]:
def getting_data(company_list, year_number, qtr, report_type):
    for i in range(0, company_list.shape[0]):
        url_address = company_list.loc[i,"url"]
        text = requests.get(url_address).text
        text = BeautifulSoup(text, "html.parser")
        target = text.find("table",{"class":"tableFile"})
        link = target.find_all("tr")[1].find_all("a")[0]["href"]
        final_link = "https://www.sec.gov"+link
        text_report = requests.get(final_link).text
        text_report = BeautifulSoup(text_report, "html.parser")
        content = text_report.text
        content = content.encode(encoding='UTF-8')
        company_list.loc[i,"full_text"] = content
    company_list["year"] = year_number
    company_list["quarter"] = qtr
    company_list["report_type"] = report_type
    return company_list

In [24]:
def putting_together(year_number, qtr, company_name, report_type, runtime=True):
    start = datetime.datetime.now()
    file_path = str(year_number) + "_QTR" + str(qtr) + "_crawler.idx"
    if not os.path.isfile(file_path):
        file_path = getting_crawler(year_number,qtr)
        print "downloaded idx file"
    #print "file already exist"
    data_start_num = milestone_search(file_path)
    data_frame = convert_to_pandas(file_path, data_start_num)
    results = search_company_name(data_frame, company_name, report_type)
    final_data = getting_data(results, year_number, qtr, report_type)
    end = datetime.datetime.now()
    final_data["date"] = pd.to_datetime(final_data["date"])
    print "DataFrame rows: " + str(final_data.shape[0])
    if runtime:
        print end - start
    return final_data

Saving the final_dataset to a pickle file, and reading in a pickle file

In [25]:
def save_as_pickle(path_to_save, final_data):
    final_data.to_pickle(path_to_save)

In [26]:
def read_data(path_to_open):
    infile = open(path_to_open,"rb")
    newdata = pickle.load(infile)
    infile.close()
    return newdata

Demonstration - extracting the latest document for all 30 companies

In [28]:
data_2017_qrt2 = putting_together(2017, 2, "sp500_new.csv","10-K")

DataFrame rows: 16
0:00:59.182327


In [29]:
data_2017_qrt1 = putting_together(2017, 1, "sp500_new.csv","10-K")

DataFrame rows: 401
0:32:35.485005


In [30]:
data_2016_qrt4 = putting_together(2016, 4, "sp500_new.csv","10-K")

DataFrame rows: 44
0:02:42.324531


In [31]:
data_2016_qrt3 = putting_together(2016, 3, "sp500_new.csv","10-K")

DataFrame rows: 29
0:01:44.636659


In [32]:
latest_sp500 = data_2017_qrt2.append([data_2017_qrt1, data_2016_qrt4, data_2016_qrt3]).reset_index(drop=True)

In [34]:
latest_sp500.to_csv("latest_sp500.csv",index=False)

In [127]:
#final.groupby(by ="CIK #")["Name"].agg("count")
final[(final["CIK #"]== "806085") & (final["year"]==2007)]

,CIK #,Name,company,date,full_text,quarter,report,report_type,url,year
20,806085,Lehman Brothers,LEHMAN BROTHERS HOLDINGS INC,2007-02-13,\n10-K\n1\na07-4192_110k.htm\n10-K\n\n\n\n\n\n...,1,10-K,10-K,https://www.sec.gov/Archives/edgar/data/806085...,2007


In [277]:
final.head(3)

NameError: name 'final' is not defined

Downloading all idx files to create a database for all report from year 2000 to 2017 qtr2 to facilitate data extraction

In [94]:
def download(year_number, qtr, runtime=True):
    start = datetime.datetime.now()
    file_path = str(year_number) + "_QTR" + str(qtr) + "_crawler.idx"
    if not os.path.isfile(file_path):
        file_path = getting_crawler(year_number,qtr)
        print "downloaded idx file"
    else:
        print "file already exists"
    if runtime:
        end = datetime.datetime.now()
        print "total runtime: " + str(end-start)

In [239]:
def create_idx(year_number, qtr):
    file_path = str(year_number) + "_QTR" + str(qtr) + "_crawler.idx"
    data_start_num = milestone_search(file_path)
    data_frame = convert_to_pandas(file_path, data_start_num)
    data_frame["year"] = year_number
    data_frame["qtr"] = qtr
    return data_frame

In [240]:
download(2017, 1)
download(2017, 2)
idx_2017_qtr1 = create_idx(2017, 1)
idx_2017_qtr2 = create_idx(2017, 2)

file already exists
total runtime: 0:00:00.000152
file already exists
total runtime: 0:00:00.000114


In [241]:
download(2016,4)
download(2016,3)
download(2016,2)
download(2016,1)
idx_2016_qtr4 = create_idx(2016, 4)
idx_2016_qtr3 = create_idx(2016, 3)
idx_2016_qtr2 = create_idx(2016, 2)
idx_2016_qtr1 = create_idx(2016, 1)

file already exists
total runtime: 0:00:00.000176
file already exists
total runtime: 0:00:00.000167
file already exists
total runtime: 0:00:00.000204
file already exists
total runtime: 0:00:00.000040


In [242]:
download(2015,4)
download(2015,3)
download(2015,2)
download(2015,1)
idx_2015_qtr4 = create_idx(2015, 4)
idx_2015_qtr3 = create_idx(2015, 3)
idx_2015_qtr2 = create_idx(2015, 2)
idx_2015_qtr1 = create_idx(2015, 1)

file already exists
total runtime: 0:00:00.000544
file already exists
total runtime: 0:00:00.000073
file already exists
total runtime: 0:00:00.000048
file already exists
total runtime: 0:00:00.000044


In [243]:
download(2014,4)
download(2014,3)
download(2014,2)
download(2014,1)
idx_2014_qtr4 = create_idx(2014, 4)
idx_2014_qtr3 = create_idx(2014, 3)
idx_2014_qtr2 = create_idx(2014, 2)
idx_2014_qtr1 = create_idx(2014, 1)

file already exists
total runtime: 0:00:00.000187
file already exists
total runtime: 0:00:00.000146
file already exists
total runtime: 0:00:00.000051
file already exists
total runtime: 0:00:00.000040


In [244]:
download(2013,4)
download(2013,3)
download(2013,2)
download(2013,1)
idx_2013_qtr4 = create_idx(2013, 4)
idx_2013_qtr3 = create_idx(2013, 3)
idx_2013_qtr2 = create_idx(2013, 2)
idx_2013_qtr1 = create_idx(2013, 1)

file already exists
total runtime: 0:00:00.000130
file already exists
total runtime: 0:00:00.000052
file already exists
total runtime: 0:00:00.000039
file already exists
total runtime: 0:00:00.000053


In [245]:
download(2012,4)
download(2012,3)
download(2012,2)
download(2012,1)
idx_2012_qtr4 = create_idx(2012, 4)
idx_2012_qtr3 = create_idx(2012, 3)
idx_2012_qtr2 = create_idx(2012, 2)
idx_2012_qtr1 = create_idx(2012, 1)

file already exists
total runtime: 0:00:00.000127
file already exists
total runtime: 0:00:00.000051
file already exists
total runtime: 0:00:00.000041
file already exists
total runtime: 0:00:00.000039


In [246]:
download(2011,4)
download(2011,3)
download(2011,2)
download(2011,1)
idx_2011_qtr4 = create_idx(2011, 4)
idx_2011_qtr3 = create_idx(2011, 3)
idx_2011_qtr2 = create_idx(2011, 2)
idx_2011_qtr1 = create_idx(2011, 1)

file already exists
total runtime: 0:00:00.000148
file already exists
total runtime: 0:00:00.000053
file already exists
total runtime: 0:00:00.000040
file already exists
total runtime: 0:00:00.000037


In [247]:
download(2010,4)
download(2010,3)
download(2010,2)
download(2010,1)
idx_2010_qtr4 = create_idx(2010, 4)
idx_2010_qtr3 = create_idx(2010, 3)
idx_2010_qtr2 = create_idx(2010, 2)
idx_2010_qtr1 = create_idx(2010, 1)

file already exists
total runtime: 0:00:00.000255
file already exists
total runtime: 0:00:00.000091
file already exists
total runtime: 0:00:00.000042
file already exists
total runtime: 0:00:00.000096


In [248]:
download(2009,4)
download(2009,3)
download(2009,2)
download(2009,1)
idx_2009_qtr4 = create_idx(2009, 4)
idx_2009_qtr3 = create_idx(2009, 3)
idx_2009_qtr2 = create_idx(2009, 2)
idx_2009_qtr1 = create_idx(2009, 1)

file already exists
total runtime: 0:00:00.000121
file already exists
total runtime: 0:00:00.000047
file already exists
total runtime: 0:00:00.000041
file already exists
total runtime: 0:00:00.000037


In [249]:
download(2008,4)
download(2008,3)
download(2008,2)
download(2008,1)
idx_2008_qtr4 = create_idx(2008, 4)
idx_2008_qtr3 = create_idx(2008, 3)
idx_2008_qtr2 = create_idx(2008, 2)
idx_2008_qtr1 = create_idx(2008, 1)

file already exists
total runtime: 0:00:00.000137
file already exists
total runtime: 0:00:00.000041
file already exists
total runtime: 0:00:00.000035
file already exists
total runtime: 0:00:00.000037


In [250]:
download(2007,4)
download(2007,3)
download(2007,2)
download(2007,1)
idx_2007_qtr4 = create_idx(2007, 4)
idx_2007_qtr3 = create_idx(2007, 3)
idx_2007_qtr2 = create_idx(2007, 2)
idx_2007_qtr1 = create_idx(2007, 1)

file already exists
total runtime: 0:00:00.000122
file already exists
total runtime: 0:00:00.000048
file already exists
total runtime: 0:00:00.000040
file already exists
total runtime: 0:00:00.000039


In [251]:
download(2006,4)
download(2006,3)
download(2006,2)
download(2006,1)
idx_2006_qtr4 = create_idx(2006, 4)
idx_2006_qtr3 = create_idx(2006, 3)
idx_2006_qtr2 = create_idx(2006, 2)
idx_2006_qtr1 = create_idx(2006, 1)

file already exists
total runtime: 0:00:00.000229
file already exists
total runtime: 0:00:00.000051
file already exists
total runtime: 0:00:00.000037
file already exists
total runtime: 0:00:00.000037


In [252]:
download(2005,4)
download(2005,3)
download(2005,2)
download(2005,1)
idx_2005_qtr4 = create_idx(2005, 4)
idx_2005_qtr3 = create_idx(2005, 3)
idx_2005_qtr2 = create_idx(2005, 2)
idx_2005_qtr1 = create_idx(2005, 1)

file already exists
total runtime: 0:00:00.000549
file already exists
total runtime: 0:00:00.000058
file already exists
total runtime: 0:00:00.000040
file already exists
total runtime: 0:00:00.000037


In [253]:
download(2004,4)
download(2004,3)
download(2004,2)
download(2004,1)
idx_2004_qtr4 = create_idx(2004, 4)
idx_2004_qtr3 = create_idx(2004, 3)
idx_2004_qtr2 = create_idx(2004, 2)
idx_2004_qtr1 = create_idx(2004, 1)

file already exists
total runtime: 0:00:00.000118
file already exists
total runtime: 0:00:00.000055
file already exists
total runtime: 0:00:00.000054
file already exists
total runtime: 0:00:00.000048


In [254]:
download(2003,4)
download(2003,3)
download(2003,2)
download(2003,1)
idx_2003_qtr4 = create_idx(2003, 4)
idx_2003_qtr3 = create_idx(2003, 3)
idx_2003_qtr2 = create_idx(2003, 2)
idx_2003_qtr1 = create_idx(2003, 1)

file already exists
total runtime: 0:00:00.000119
file already exists
total runtime: 0:00:00.000048
file already exists
total runtime: 0:00:00.000042
file already exists
total runtime: 0:00:00.000038


In [255]:
download(2002,4)
download(2002,3)
download(2002,2)
download(2002,1)
idx_2002_qtr4 = create_idx(2002, 4)
idx_2002_qtr3 = create_idx(2002, 3)
idx_2002_qtr2 = create_idx(2002, 2)
idx_2002_qtr1 = create_idx(2002, 1)

file already exists
total runtime: 0:00:00.000116
file already exists
total runtime: 0:00:00.000043
file already exists
total runtime: 0:00:00.000037
file already exists
total runtime: 0:00:00.001303


In [256]:
download(2001,4)
download(2001,3)
download(2001,2)
download(2001,1)
idx_2001_qtr4 = create_idx(2001, 4)
idx_2001_qtr3 = create_idx(2001, 3)
idx_2001_qtr2 = create_idx(2001, 2)
idx_2001_qtr1 = create_idx(2001, 1)

file already exists
total runtime: 0:00:00.000124
file already exists
total runtime: 0:00:00.000050
file already exists
total runtime: 0:00:00.000043
file already exists
total runtime: 0:00:00.000038


In [257]:
download(2000,4)
download(2000,3)
download(2000,2)
download(2000,1)
idx_2000_qtr4 = create_idx(2000, 4)
idx_2000_qtr3 = create_idx(2000, 3)
idx_2000_qtr2 = create_idx(2000, 2)
idx_2000_qtr1 = create_idx(2000, 1)

file already exists
total runtime: 0:00:00.000110
file already exists
total runtime: 0:00:00.000040
file already exists
total runtime: 0:00:00.000038
file already exists
total runtime: 0:00:00.000038


In [137]:
listing = [idx_2017_qtr2, idx_2017_qtr1, idx_2016_qtr4, idx_2016_qtr3, idx_2016_qtr2, idx_2016_qtr1,
          idx_2015_qtr4, idx_2015_qtr3, idx_2015_qtr2, idx_2015_qtr1, 
           idx_2014_qtr4, idx_2014_qtr3, idx_2014_qtr2, idx_2014_qtr1,
          idx_2013_qtr4, idx_2013_qtr3, idx_2013_qtr2, idx_2013_qtr1,
          idx_2012_qtr4, idx_2012_qtr3, idx_2012_qtr2, idx_2012_qtr1,
          idx_2011_qtr4, idx_2011_qtr3, idx_2011_qtr2, idx_2011_qtr1,
          idx_2010_qtr4, idx_2010_qtr3, idx_2010_qtr2, idx_2010_qtr1,
          idx_2009_qtr4, idx_2009_qtr3, idx_2009_qtr2, idx_2009_qtr1,
          idx_2008_qtr4, idx_2008_qtr3, idx_2008_qtr2, idx_2008_qtr1,
          idx_2007_qtr4, idx_2007_qtr3, idx_2007_qtr2, idx_2007_qtr1,
           idx_2006_qtr4, idx_2006_qtr3, idx_2006_qtr2, idx_2006_qtr1,
          idx_2005_qtr4, idx_2005_qtr3, idx_2005_qtr2, idx_2005_qtr1,
          idx_2004_qtr4, idx_2004_qtr3, idx_2004_qtr2, idx_2004_qtr1,
          idx_2003_qtr4, idx_2003_qtr3, idx_2003_qtr2, idx_2003_qtr1,
          idx_2002_qtr4, idx_2002_qtr3, idx_2002_qtr2, idx_2002_qtr1,
          idx_2001_qtr4, idx_2001_qtr3, idx_2001_qtr2, idx_2001_qtr1,
          idx_2000_qtr4, idx_2000_qtr3, idx_2000_qtr2, idx_2000_qtr1]

In [143]:
all_listing = pd.concat(listing, axis=0)

In [147]:
all_listing.shape

(15246085, 6)

In [382]:
all_listing.head(5)

,CIK,company,date,report,url,company_l
0,1703629,'Ohana Labs Inc.,2017-04-12,D,https://www.sec.gov/Archives/edgar/data/170362...,'ohana labs inc.
1,1307969,"024 Pharma, Inc.",2017-04-11,10-Q,https://www.sec.gov/Archives/edgar/data/130796...,"024 pharma, inc."
2,1307969,"024 Pharma, Inc.",2017-05-11,10-Q/A,https://www.sec.gov/Archives/edgar/data/130796...,"024 pharma, inc."
3,1084869,1 800 FLOWERS COM INC,2017-05-12,10-Q,https://www.sec.gov/Archives/edgar/data/108486...,1 800 flowers com inc
4,1084869,1 800 FLOWERS COM INC,2017-05-24,4,https://www.sec.gov/Archives/edgar/data/108486...,1 800 flowers com inc


In [219]:
cleaned = all_listing[all_listing['date'].str.contains("-")].reset_index(drop=True)

In [225]:
def find_text(x):
    if re.search("\d{4}-\d{2}-\d{2}",x):
        return 1
    else:
        return 0

In [226]:
cleaned["date_clean"] = cleaned["date"].apply(lambda y: find_text(y))

In [228]:
cleaned["date_clean"].value_counts()

1    15170492
0         553
Name: date_clean, dtype: int64

In [230]:
cleaned = cleaned[cleaned["date_clean"]==1]

In [232]:
cleaned["date"] = pd.to_datetime(cleaned["date"],format = "%Y-%m-%d")

/Users/jasonchiu0803/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [238]:
cleaned[cleaned["report"]=="10-K"].shape

(144032, 7)

In [398]:
cleaned["company"] = cleaned["company"].str.replace(",","")

In [399]:
cleaned["company_l"] = cleaned["company_l"].str.replace(",","")

In [402]:
cleaned = cleaned[["CIK","company","date","report","url","company_l"]]

In [403]:
cleaned.to_csv("idx_2000_2017_qtr2.csv",index= False)

Writing a function to match company name to CIK # in the full list.

In [278]:
new_name = pd.read_csv("Name.csv")

In [279]:
new_name.head(2)

,id,Ticker,Name,Sector,Industry,Headquarter,Country,Currency
0,1,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",USA,USD
1,2,ABT,Abbott Laboratories,Health Care,Health Care Equipment & Services,"North Chicago, Illinois",USA,USD


In [280]:
new_name["Name_lower"] = new_name["Name"].str.lower()

In [281]:
new_name.head(2)

,id,Ticker,Name,Sector,Industry,Headquarter,Country,Currency,Name_lower
0,1,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",USA,USD,3m company
1,2,ABT,Abbott Laboratories,Health Care,Health Care Equipment & Services,"North Chicago, Illinois",USA,USD,abbott laboratories


In [283]:
USA_only = new_name[new_name["Country"]=="USA"]

In [284]:
USA_only.head(2)

,id,Ticker,Name,Sector,Industry,Headquarter,Country,Currency,Name_lower
0,1,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",USA,USD,3m company
1,2,ABT,Abbott Laboratories,Health Care,Health Care Equipment & Services,"North Chicago, Illinois",USA,USD,abbott laboratories


In [292]:
# first merge:
merged = pd.merge(USA_only, cik_nm, how='left', left_on="Name_lower", right_on="Name",
         suffixes=('_x', '_y'))

In [293]:
merged.shape

(3704, 13)

In [295]:
sum(pd.isnull(merged["CIK #"]))

3166

In [298]:
# replaceing Company with co
USA_only["Name_lower"] = USA_only["Name_lower"].str.replace("company", "co")

/Users/jasonchiu0803/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [300]:
#second merge
merged = pd.merge(USA_only, cik_nm, how='left', left_on="Name_lower", right_on="Name",
         suffixes=('_x', '_y'))

In [302]:
sum(pd.isnull(merged["CIK #"]))

3161

In [332]:
cik_nm.head(1)

,company,Name,cik #,CIK #
0,!J INC:0001438823:,!j inc,0001438823,1438823


In [424]:
USA_only.head(1)

,id,Ticker,Name,Sector,Industry,Headquarter,Country,Currency,Name_lower
0,1,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",USA,USD,3m co


In [ ]:
USA_only

In [418]:
def matching_name(x):
    for i in range(x.shape[0]):
        if pd.isnull(x.loc[i,"cik #"]):
            company_name = x.loc[i, "Name_lower"]
            a = cik_nm[cik_nm["Name"].str.contains(company_name)].reset_index(drop=True)
            new_name_inc = x.loc[i, "Name_lower"] + " inc"
            new_name = x.loc[i, "Name_lower"].translate(None, string.punctuation)
            new_name_inc_period = x.loc[i, "Name_lower"] + " inc."
            new_name_inc_comma = x.loc[i, "Name_lower"] + ", inc."
            new_name_de = x.loc[i, "Name_lower"] + " /DE/"
            new_name_plc = x.loc[i, "Name_lower"] + " plc"
            new_name_space = re.sub(",","",x.loc[i, "Name_lower"])
            new_name_space_peroid = re.sub("."," ",x.loc[i, "Name_lower"])
            new_name_no_class = re.sub("class a", "", x.loc[i, "Name_lower"])
            print new_name_no_class
            new_name_no_classB = re.sub("class b", "", x.loc[i, "Name_lower"])
            print new_name_no_classB
            print str(i) + " word"
            #print new_name_inc
            #print new_name_inc_period
            #print a.shape
            if a.empty:
                print "no data"
            else:
                for j in range(a.shape[0]):
                    if a.loc[j, "Name"] == new_name_inc_comma:
                        x.loc[i, "company"] = a.loc[j,"company"]
                        x.loc[i, "Name_y"] = a.loc[j,"Name"]
                        x.loc[i, "cik #"] = a.loc[j,"cik #"]
                        x.loc[i, "CIK #"] = a.loc[j,"CIK #"]
                        print "found it"
                    if a.loc[j, "Name"] == new_name_de:
                        x.loc[i, "company"] = a.loc[j,"company"]
                        x.loc[i, "Name_y"] = a.loc[j,"Name"]
                        x.loc[i, "cik #"] = a.loc[j,"cik #"]
                        x.loc[i, "CIK #"] = a.loc[j,"CIK #"]
                        print "found it"
                    if a.loc[j, "Name"] == new_name_plc:
                        x.loc[i, "company"] = a.loc[j,"company"]
                        x.loc[i, "Name_y"] = a.loc[j,"Name"]
                        x.loc[i, "cik #"] = a.loc[j,"cik #"]
                        x.loc[i, "CIK #"] = a.loc[j,"CIK #"]
                        print "found it"
                    if a.loc[j, "Name"] == new_name_space:
                        x.loc[i, "company"] = a.loc[j,"company"]
                        x.loc[i, "Name_y"] = a.loc[j,"Name"]
                        x.loc[i, "cik #"] = a.loc[j,"cik #"]
                        x.loc[i, "CIK #"] = a.loc[j,"CIK #"]
                        print "found it"
                    if a.loc[j, "Name"] == new_name_no_class:
                        x.loc[i, "company"] = a.loc[j,"company"]
                        x.loc[i, "Name_y"] = a.loc[j,"Name"]
                        x.loc[i, "cik #"] = a.loc[j,"cik #"]
                        x.loc[i, "CIK #"] = a.loc[j,"CIK #"]
                        print "found it"
                    if a.loc[j, "Name"] == new_name_no_classB:
                        x.loc[i, "company"] = a.loc[j,"company"]
                        x.loc[i, "Name_y"] = a.loc[j,"Name"]
                        x.loc[i, "cik #"] = a.loc[j,"cik #"]
                        x.loc[i, "CIK #"] = a.loc[j,"CIK #"]
                        print "found it"
                    if a.loc[j, "Name"] == new_name_space_peroid:
                        x.loc[i, "company"] = a.loc[j,"company"]
                        x.loc[i, "Name_y"] = a.loc[j,"Name"]
                        x.loc[i, "cik #"] = a.loc[j,"cik #"]
                        x.loc[i, "CIK #"] = a.loc[j,"CIK #"]
                        print "found it"
                    if a.loc[j, "Name"].translate(None, string.punctuation) == new_name_inc_comma:
                        x.loc[i, "company"] = a.loc[j,"company"]
                        x.loc[i, "Name_y"] = a.loc[j,"Name"]
                        x.loc[i, "cik #"] = a.loc[j,"cik #"]
                        x.loc[i, "CIK #"] = a.loc[j,"CIK #"]
                        print "found it"

                    #if a.loc[j, "Name"] == new_name_inc:
                        #x.loc[i, "company"] = a.loc[j,"company"]
                        #x.loc[i, "Name_y"] = a.loc[j,"Name"]
                        #x.loc[i, "cik #"] = a.loc[j,"cik #"]
                        #x.loc[i, "CIK #"] = a.loc[j,"CIK #"]
                        #print "found it" 
                        #continue
                    #if a.loc[j, "Name"] == new_name_inc_period:
                        #x.loc[i, "company"] = a.loc[j,"company"]
                        #x.loc[i, "Name_y"] = a.loc[j,"Name"]
                        #x.loc[i, "cik #"] = a.loc[j,"cik #"]
                        #x.loc[i, "CIK #"] = a.loc[j,"CIK #"]
                        #print "found it"

In [419]:
matching_name(merged)

air products & chemicals inc
air products & chemicals inc
13 word
alliance data systems
alliance data systems
23 word
alphabet inc 
alphabet inc class a
25 word
no data
alphabet inc class c
alphabet inc class c
26 word
no data
amazon.com inc
amazon.com inc
28 word
american electric power
american electric power
31 word
american international group, inc.
american international group, inc.
33 word
american tower corp a
american tower corp a
34 word
no data
amphenol corp a
amphenol corp a
40 word
no data
analog devices, inc.
analog devices, inc.
42 word
no data
apache corporation
apache corporation
44 word
no data
apartment investment & mgmt
apartment investment & mgmt
45 word
no data
apple inc.
apple inc.
46 word
archer-daniels-midland co
archer-daniels-midland co
48 word
no data
arthur j. gallagher & co.
arthur j. gallagher & co.
49 word
no data
at&t inc
at&t inc
51 word
avago technologies
avago technologies
56 word
avalonbay communities, inc.
avalonbay communities, inc.
57 word
no data

/Users/jasonchiu0803/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


bard (c.r.) inc.
bard (c.r.) inc.
64 word
no data
baxter international inc.
baxter international inc.
65 word
bb&t corporation
bb&t corporation
66 word
no data
becton dickinson
becton dickinson
67 word
best buy co. inc.
best buy co. inc.
70 word
no data
block h&r
block h&r
73 word
no data
boston scientific
boston scientific
77 word
bristol-myers squibb
bristol-myers squibb
78 word
no data
brown-forman corporation
brown-forman corporation
79 word
no data
c. h. robinson worldwide
c. h. robinson worldwide
80 word
no data
cabot oil & gas
cabot oil & gas
82 word
campbell soup
campbell soup
83 word
capital one financial
capital one financial
84 word
cardinal health inc.
cardinal health inc.
85 word
no data
carnival corp.
carnival corp.
88 word
no data
caterpillar inc.
caterpillar inc.
89 word
no data
cbs corp.
cbs corp.
91 word
no data
celgene corp.
celgene corp.
92 word
centene corporation
centene corporation
93 word
no data
centurylink inc
centurylink inc
95 word
no data
cerner
cerner
96 w

KeyboardInterrupt: 

In [423]:
sum(pd.isnull(merged["CIK #"]))

3004

In [408]:
merged[pd.isnull(merged["CIK #"])]

,id,Ticker,Name_x,Sector,Industry,Headquarter,Country,Currency,Name_lower,company,Name_y,cik #,CIK #
13,14,APD,Air Products & Chemicals Inc,Materials,Industrial Gases,"Allentown, Pennsylvania",USA,USD,air products & chemicals inc,NaN,NaN,NaN,NaN
23,23,ADS,Alliance Data Systems,Information Technology,Data Processing & Outsourced Services,"Plano, Texas",USA,USD,alliance data systems,NaN,NaN,NaN,NaN
25,25,GOOGL,Alphabet Inc Class A,Information Technology,Internet Software & Services,"Mountain View, California",USA,USD,alphabet inc class a,NaN,NaN,NaN,NaN
26,26,GOOG,Alphabet Inc Class C,Information Technology,Internet Software & Services,"Mountain View, California",USA,USD,alphabet inc class c,NaN,NaN,NaN,NaN
28,28,AMZN,Amazon.com Inc,Consumer Discretionary,Internet Retail,"Seattle, Washington",USA,USD,amazon.com inc,NaN,NaN,NaN,NaN
31,31,AEP,American Electric Power,Utilities,Electric Utilities,"Columbus, Ohio",USA,USD,american electric power,NaN,NaN,NaN,NaN
33,33,AIG,"American International Group, Inc.",Financials,Property & Casualty Insurance,"New York, New York",USA,USD,"american international group, inc.",NaN,NaN,NaN,NaN
34,34,AMT,American Tower Corp A,Financials,Specialized REITs,"Boston, Massachusetts",USA,USD,american tower corp a,NaN,NaN,NaN,NaN
40,40,APH,Amphenol Corp A,Industrials,Electrical Components & Equipment,"Wallingford, Connecticut",USA,USD,amphenol corp a,NaN,NaN,NaN,NaN
42,42,ADI,"Analog Devices, Inc.",Information Technology,Semiconductors,"Norwood, Massachusetts",USA,USD,"analog devices, inc.",NaN,NaN,NaN,NaN


In [394]:
cik_nm[cik_nm["Name"].str.contains("amazon")]

,company,Name,cik #,CIK #
26218,AMAZON 13-30 LP:0001567126:,amazon 13-30 lp,0001567126,1567126
26219,AMAZON BIOTECH INC:0001088781:,amazon biotech inc,0001088781,1088781
26220,AMAZON COM INC:0001018724:,amazon com inc,0001018724,1018724
26221,AMAZON FORMS ONE INC:0001276224:,amazon forms one inc,0001276224,1276224
26222,AMAZON GOLDSANDS LTD.:0001045929:,amazon goldsands ltd.,0001045929,1045929
26223,AMAZON HERB CO:0000939780:,amazon herb co,0000939780,93978
26224,AMAZON MARKET NEUTRAL FUND:0001671944:,amazon market neutral fund,0001671944,1671944
26225,AMAZON MINING HOLDING PLC:0001420649:,amazon mining holding plc,0001420649,1420649
26226,AMAZON NATURAL TREASURES COM INC ...,amazon natural treasures com inc ...,0001099200,10992
26227,AMAZON NATURAL TREASURES INC:0000844055:,amazon natural treasures inc,0000844055,844055


In [319]:
#replacing 
USA_only.head(2)

,id,Ticker,Name,Sector,Industry,Headquarter,Country,Currency,Name_lower
0,1,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",USA,USD,3m co
1,2,ABT,Abbott Laboratories,Health Care,Health Care Equipment & Services,"North Chicago, Illinois",USA,USD,abbott laboratories


'gjeodkakpgejojakfepkf'

In [89]:
merged = pd.merge(new_name, cik_nm, how='left', left_on="Name_lower", right_on="Name",
         suffixes=('_x', '_y'))

In [90]:
merged.shape

(4275, 13)

In [266]:
USA_only = merged[merged["Country"]=="USA"]

In [269]:
USA_only

,id,Ticker,Name_x,Sector,Industry,Headquarter,Country,Currency,Name_lower,company,Name_y,cik #,CIK #
0,1,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",USA,USD,3m company,NaN,NaN,NaN,NaN
1,2,ABT,Abbott Laboratories,Health Care,Health Care Equipment & Services,"North Chicago, Illinois",USA,USD,abbott laboratories,ABBOTT LABORATORIES:0000001800:,abbott laboratories,0000001800,18
2,3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",USA,USD,abbvie,NaN,NaN,NaN,NaN
3,4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",USA,USD,accenture plc,ACCENTURE PLC:0001467373:,accenture plc,0001467373,1467373
4,5,ATVI,Activision Blizzard,Information Technology,Home Entertainment Software,"Santa Monica, California",USA,USD,activision blizzard,NaN,NaN,NaN,NaN
5,6,AYI,Acuity Brands Inc,Industrials,Electrical Components & Equipment,"Atlanta, Georgia",USA,USD,acuity brands inc,ACUITY BRANDS INC:0001144215:,acuity brands inc,0001144215,1144215
6,7,ADBE,Adobe Systems Inc,Information Technology,Application Software,"San Jose, California",USA,USD,adobe systems inc,ADOBE SYSTEMS INC:0000796343:,adobe systems inc,0000796343,796343
7,8,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,"Roanoke, Virginia",USA,USD,advance auto parts,NaN,NaN,NaN,NaN
8,9,AES,AES Corp,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",USA,USD,aes corp,AES CORP:0000874761:,aes corp,0000874761,874761
9,10,AET,Aetna Inc,Health Care,Managed Health Care,"Hartford, Connecticut",USA,USD,aetna inc,AETNA INC:0001013761:,aetna inc,0001013761,1013761


(490, 10)